# MINI PROJECT


### PLANT DISEASE IMAGE CLASSIFICATION

In [ ]:
#importing important libraries
# Imports
#not all used
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import img_to_array,  load_img
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
!unzip "/content/cotton.zip"

Archive:  /content/cotton.zip
replace cotton/test/diseased cotton leaf/dis_leaf (124).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pathlib
data_dir = pathlib.Path('/content/cotton/train') # make a directory "data_dir"/content/drive/MyDrive/DatesImages/DatesImages
path = Path(data_dir)
path.ls() #list all the folders of dates images.
path

Path('/content/cotton/train')

In [ ]:
import pathlib
data_dir1 = pathlib.Path('/content/cotton/test') # make a directory "data_dir"/content/drive/MyDrive/DatesImages/DatesImages
path1 = Path(data_dir1)
path1.ls() #list all the folders of dates images.
path1

Path('/content/cotton/test')

In [ ]:
def number_of_images_in_dataset(dataset):
  images = list(dataset.glob("*/*.*"))
  return len(images)
print("number of all images in dataset: {}".format(number_of_images_in_dataset(data_dir)))

number of all images in dataset: 122


In [ ]:
def number_of_images_in_dataset(dataset):
  images = list(dataset.glob("*/*.*"))
  return len(images)
print("number of all images in dataset: {}".format(number_of_images_in_dataset(data_dir1)))

number of all images in dataset: 11


### DATA PREPROCESSING

In [ ]:
# Define data preprocessing transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


### DATASET LOADING

In [ ]:
from torchvision.datasets import ImageFolder
train_dataset = ImageFolder(root=data_dir, transform=transform)
test_dataset = ImageFolder(root=data_dir1, transform=transform)

In [ ]:
# Create DataLoader for training
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=2)

In [ ]:
num_classes = 2

### BUILDING SIMPLE CNN MODEL

In [ ]:
# Define your CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 128 * 128, 64)
        self.fc2 = nn.Linear(64, num_classes)  # Assuming 'num_classes' is the number of classes in your dataset

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * 128 * 128)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Print training loss for this epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

Epoch 1, Loss: 5.323312946455992
Epoch 2, Loss: 2.321087315427263
Epoch 3, Loss: 0.9892421831056232
Epoch 4, Loss: 1.3504804669951898
Epoch 5, Loss: 1.1689230898351286
Epoch 6, Loss: 0.8935395527676847
Epoch 7, Loss: 0.8966442073302221
Epoch 8, Loss: 0.8438396052510829
Epoch 9, Loss: 0.8483576693566856
Epoch 10, Loss: 0.8245837343894862


In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on validation data: {100 * correct / total}%")

Accuracy on validation data: 90.9090909090909%


### Model Evaluation

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define the same transform used for training and validation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the sample image and apply the same transform
sample_image_path = '/content/cotton/test/fresh cotton leaf/d (61)_iaip - Copy.jpg'
sample_image = Image.open(sample_image_path)
sample_image = transform(sample_image).unsqueeze(0)  # Add a batch dimension

# Load the trained model

model = SimpleCNN()  # Create an instance of your model

model.eval()  # Set the model to evaluation mode

# Make predictions on the sample image
with torch.no_grad():
    output = model(sample_image)

# Assuming output is a tensor of shape (1, num_classes), where num_classes is the number of classes in your dataset
# You can convert it to class probabilities using softmax
import torch.nn.functional as F
probabilities = F.softmax(output, dim=1)

# Get the predicted class label
_, predicted_class = torch.max(output, 1)

# Define class names
class_names = ['Disease', 'Normal']

# Map the predicted class index to the class name
predicted_class_name = class_names[predicted_class.item()]

print(f"The Predicted Class is : {predicted_class_name}")


The Predicted Class is : Normal


In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define the same transform used for training and validation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the sample image and apply the same transform
sample_image_path = '/content/cotton/train/diseased cotton leaf/dis_leaf (24)_iaip.jpg'
sample_image = Image.open(sample_image_path)
sample_image = transform(sample_image).unsqueeze(0)  # Add a batch dimension

# Load the trained model

model = SimpleCNN()  # Create an instance of your model

model.eval()  # Set the model to evaluation mode

# Make predictions on the sample image
with torch.no_grad():
    output = model(sample_image)

# Assuming output is a tensor of shape (1, num_classes), where num_classes is the number of classes in your dataset
# You can convert it to class probabilities using softmax
import torch.nn.functional as F
probabilities = F.softmax(output, dim=1)

# Get the predicted class label
_, predicted_class = torch.max(output, 1)

# Define class names
class_names = ['Disease', 'Normal']

# Map the predicted class index to the class name
predicted_class_name = class_names[predicted_class.item()]

print(f"The Predicted Class is : {predicted_class_name}")


The Predicted Class is : Disease
